In [ ]:
import glob
import math
import uproot
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil
import uproot_methods

import function_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path1 = r'C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Theory\TFMoX-S\enant=S_hel=+1_KE=3.0eV' # use your path
path2 = r'C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Theory\TFMoX-S\enant=S_hel=+1_KE=4.0eV' # use your path
path3 = r'C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Theory\TFMoX-S\enant=S_hel=+1_KE=6.1eV' # use your path
path4 = r'C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Theory\TFMoX-S\enant=S_hel=+1_KE=8.2eV' # use your path
path = r'C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Theory\TFMoX-S\enant=S_hel=+1_KE=11.5eV' # use your path

all_files = glob.glob(path + "/*.dat")

li = []
cosphi_th = []
allnames=[]
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    cosphi_th.append((int(df["filename"].str.split("_")[0][0]),int(df["filename"].str.split("_")[0][1])))
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th=np.array(cosphi_th)
frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10
frame_srt = frame.groupby(by=["filename"],sort=False)
#this syntax has 1_1 followed by 1_10
# frame_srt_old = frame.groupby(by=["filename"])

for key, item in frame_srt:
    allnames.append(key)

frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
frame_srt.describe()

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
import matplotlib as mpl
from matplotlib import cm
import matplotlib.colors

#the following converts a cmap into plotly language 
magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

#temperature cmap
colors = [[0, "blue"],
          [0.5, "white"],
          [0.75, "yellow"],
          [1, "red"]]

cmap_temp = matplotlib.colors.LinearSegmentedColormap.from_list("temperature_philipp", colors)
cmap_temp_go = go.color_continuous_scale=[(0, "blue"), (0.5, "white"), (0.75, "yellow"), (1,"red")]

In [ ]:
#NOTE: the following phi and ctheta are not unique: len=20000
phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
phi_rad = phi_full * np.pi/180.

ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
theta_rad = np.arccos(ctheta_full)

#NOTE: the following phi and ctheta are unique: dim=(200,100)
# it is already the middle point, no need of doing it again
phiM = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
cosM = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

#derivatio from previous definition
# phiM=(phi[1:] + phi[:-1])/2
# cosM=(ctheta[1:] + ctheta[:-1])/2

#cartesian product for the correct representation of b1 maps
#has to be adapted to the THEORY: 0.835,-0.835,-165,165
#accordinf to automatic_72_CPR.f90 from philipp [1,6] is from [cos(0), cos(pi)] = [1,-1]

# phicos_MFPAD = np.array(list(itertools.product(np.linspace(-180,180,12).tolist(), np.linspace(1,-1,6).tolist())))
phicos_MFPAD = np.array(list(itertools.product(np.linspace(-165,165,12).tolist(), np.linspace(0.835,-0.835,6).tolist())))
go_phi=[col[0] for col in phicos_MFPAD]
go_cos=[col[1] for col in phicos_MFPAD]

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print(type(phiM), type(cosM))


counts_1_1 = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values
counts_1_1 = counts_1_1.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_1_1[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_1_1[1,:]), "type ", type(counts_1_1))

In [ ]:
#manual definiton of light vector in MF. The order is the one matching the MFPADs of Philipp
#according to automatic_72_CPR.f90 from philipp [1,6] is from [cos(0), cos(pi)] = [1,-1]

#SHOULD I SORT IT COS PHI TOO?
cosphi_adj=list(fugi.remap(cosphi_th,0.835,-0.835,-165,165))
# cosphi_adj=list(remap(cosphi_th,-1.,1.,-180,180))
cosphi_adj=np.around(cosphi_adj,3)

#convertion to cartesian coordinate in MF
theta_phot_rad = np.arccos(cosphi_adj[0][0])
phi_phot_rad = cosphi_adj[0][1] * np.pi/180.

# theta_rad = np.arccos(ctheta_fixed)
# phi_rad = phi_fixed * np.pi/180.

count_phot = 1
# countsn = counts_fixed

print(cosphi_adj[0][0])

#look to the values in b1 map and found the MFPAD

dfind=pd.DataFrame(cosphi_adj, columns=["ctheta","phi"])
#the assignment of this colums depens on frame_str to be SORTED or UNSORTED
#the most straightforward (ad right) aproach is UNSORTED -> 1_1, 1_2 instead of 1_1, 1_10
dfind["item"]=allnames

cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values

#rearranging the photon vector
cosphi_adj_cos=np.array((dfind.sort_values(by=["ctheta","phi"]))[["ctheta","phi"]].values)
cosphi_adj_phi=np.array((dfind.sort_values(by=["phi","ctheta"]))[["ctheta","phi"]].values)

# cosphi_adj_cos
dfind.sort_values(by=["phi", "ctheta"])
dfind.loc[dfind["item"] == "4_7"]

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_1_1.T,extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')


fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phiM,cosM,counts_1_1.T, levels=20, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
fig1.colorbar(CS)

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
from numpy.core.defchararray import multiply

counts_temp=[] #standard cos sorted S enantiomer
MFPAD_S_phi=[]
MFPAD_S_cos=[]
MFPAD_R_phi=[]
MFPAD_R_cos=[]

#respect to cs is the standard calculation
cos_projthS_phi=[]
phi_projthS_phi=[]

cos_projthR_phi=[]
phi_projthR_phi=[]

cos_projthS_cos=[]
phi_projthS_cos=[]

cos_projthR_cos=[]
phi_projthR_cos=[]

#To have the right matching of MFPADs array and allnames frame_srt shouldn't be UNSORTED (see first cell)
#if UNSORTED MFPAD[0] = filename 1_1; MFPAD[1] = filename 1_2 
#if SORTED MFPAD[0] = filename 1_1; MFPAD[1] = filename 1_10 
for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    counts_temp.append(group[1]["value"].to_numpy().reshape(200,100))

#sorting: MFPAD are already the type np.array    
counts_temp=np.array(counts_temp)
MFPAD_S_phi=fugi.sorting_array(counts_temp, cosphi_th, phiM, cosM, 2) #sorted according to phi
MFPAD_S_cos=fugi.sorting_array(counts_temp, cosphi_th, phiM, cosM, 1) #sorted according to cos

for j,el in enumerate(counts_temp):
    #integration of MFPAD onto cos or phi for S enantiomer 
    cos_projthS_phi.append(MFPAD_S_phi[j].sum(axis=0))
    phi_projthS_phi.append(MFPAD_S_phi[j].sum(axis=1))
    cos_projthS_cos.append(MFPAD_S_cos[j].sum(axis=0))
    phi_projthS_cos.append(MFPAD_S_cos[j].sum(axis=1))

    #flipping for R enantiomer and sorting
    tempcos_phi=np.flip(MFPAD_S_phi[j], axis=0)
    MFPAD_R_phi.append(np.flip(tempcos_phi, axis=1))

    tempcos_cos=np.flip(MFPAD_S_cos[j], axis=0)
    MFPAD_R_cos.append(np.flip(tempcos_cos, axis=1))
    
    #integration of of MFPAD onto cos or phi for R enantiomer
    cos_projthR_phi.append(MFPAD_R_phi[j].sum(axis=0))
    phi_projthR_phi.append(MFPAD_R_phi[j].sum(axis=1))
    cos_projthR_cos.append(MFPAD_R_cos[j].sum(axis=0))
    phi_projthR_cos.append(MFPAD_R_cos[j].sum(axis=1))


#converting counts into np.array
MFPAD_R_phi=np.array(MFPAD_R_phi)
MFPAD_R_cos=np.array(MFPAD_R_cos)

#converting integrations to np.array
cos_projthS_phi=np.array(cos_projthS_phi)
phi_projthS_phi=np.array(phi_projthS_phi)
cos_projthR_phi=np.array(cos_projthR_phi)
phi_projthR_phi=np.array(phi_projthR_phi)

cos_projthS_cos=np.array(cos_projthS_cos)
phi_projthS_cos=np.array(phi_projthS_cos)
cos_projthR_cos=np.array(cos_projthR_cos)
phi_projthR_cos=np.array(phi_projthR_cos)

print(np.array(MFPAD_S_phi).shape)
print(np.array(MFPAD_S_cos).shape)
print(np.array(MFPAD_R_phi).shape)
print(np.array(MFPAD_R_cos).shape)
print(np.array(cos_projthS_phi).shape)
print(np.array(cos_projthR_phi).shape)

In [ ]:
MFPAD_S_cos_norm=[]
MFPAD_S_phi_norm=[]
MFPAD_R_cos_norm=[]
MFPAD_R_phi_norm=[]
mPECD_cos=[]
mPECD_phi=[]

#///////////////////////// cos ///////////////////////////////////
for cols,colr in zip(MFPAD_S_cos,MFPAD_R_cos):
    MFPAD_S_cos_norm.append(cols/cols.sum(axis=1)[:,np.newaxis])
    MFPAD_R_cos_norm.append(colr/colr.sum(axis=1)[:,np.newaxis])

MFPAD_S_cos_norm=np.array(MFPAD_S_cos_norm)
MFPAD_R_cos_norm=np.array(MFPAD_R_cos_norm)
MFPAD_S_cos_norm_sum=np.sum(MFPAD_S_cos_norm,axis=0)
MFPAD_R_cos_norm_sum=np.sum(MFPAD_R_cos_norm,axis=0)

mPECD_cos=np.divide(np.subtract(MFPAD_R_cos_norm,MFPAD_S_cos_norm),np.add(MFPAD_R_cos_norm,MFPAD_S_cos_norm))
mPECD_cos_sum=np.divide(np.subtract(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum),np.add(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum))

#///////////////////////// phi ///////////////////////////////////
for colsp,colrp in zip(MFPAD_S_phi,MFPAD_R_phi):
    MFPAD_S_phi_norm.append(colsp/colsp.sum(axis=1)[:,np.newaxis])
    MFPAD_R_phi_norm.append(colrp/colrp.sum(axis=1)[:,np.newaxis])

MFPAD_S_phi_norm=np.array(MFPAD_S_phi_norm)
MFPAD_R_phi_norm=np.array(MFPAD_R_phi_norm)
MFPAD_S_phi_norm_sum=np.sum(MFPAD_S_phi_norm,axis=0)
MFPAD_R_phi_norm_sum=np.sum(MFPAD_R_phi_norm,axis=0)

#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

mPECD_phi=np.divide(np.subtract(MFPAD_R_phi_norm,MFPAD_S_phi_norm),np.add(MFPAD_R_phi_norm,MFPAD_S_phi_norm))
mPECD_phi_sum=np.divide(np.subtract(MFPAD_R_phi_norm_sum,MFPAD_S_phi_norm_sum),np.add(MFPAD_R_phi_norm_sum,MFPAD_S_phi_norm_sum))

#NOTE: R-S nad S-R seems to have no influence on b1 map
cPECD_cos =  np.divide(np.subtract(cos_projthR_cos,cos_projthS_cos),np.add(cos_projthR_cos,cos_projthS_cos))
cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))

#it is somehow unconventional, but let's normalise the error with the square root of all counts
errPECD_cos = np.sqrt(1/(cos_projthR_cos+cos_projthS_cos))

In [ ]:
#phi and cos along z-axis by definition
cos_projthS_phi_LF=[]
phi_projthS_phi_LF=[]
cos_projthR_phi_LF=[]
phi_projthR_phi_LF=[]
cos_projthS_cos_LF=[]
phi_projthS_cos_LF=[]
cos_projthR_cos_LF=[]
phi_projthR_cos_LF=[]

MFPAD_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad,phi_rad,cosphi_adj_phi)[0]
MFPAD_R_phi_LF=fugi.rot3d_MFPAD(MFPAD_R_phi,theta_rad,phi_rad,cosphi_adj_phi)[0]
MFPAD_S_cos_LF=fugi.rot3d_MFPAD(MFPAD_S_cos,theta_rad,phi_rad,cosphi_adj_cos)[0]
MFPAD_R_cos_LF=fugi.rot3d_MFPAD(MFPAD_R_cos,theta_rad,phi_rad,cosphi_adj_cos)[0]

for j,el in enumerate(MFPAD_S_phi_LF):
    #NOTE PROPABABLY ONE NEEDS TO DEFINE A FUCNTION AND PROJECT AND FIND THE COS(THETAx)
    #LIKELY TO MODIFY THA rot3d_MFPAD function
    #integration of MFPAD onto cos or phi for S enantiomer 
    cos_projthS_phi_LF.append(MFPAD_S_phi_LF[j].sum(axis=0))
    phi_projthS_phi_LF.append(MFPAD_S_phi_LF[j].sum(axis=1))
    cos_projthS_cos_LF.append(MFPAD_S_cos_LF[j].sum(axis=0))
    phi_projthS_cos_LF.append(MFPAD_S_cos_LF[j].sum(axis=1))
  
    #integration of MFPAD onto cos or phi for R enantiomer
    cos_projthR_phi_LF.append(MFPAD_R_phi_LF[j].sum(axis=0))
    phi_projthR_phi_LF.append(MFPAD_R_phi_LF[j].sum(axis=1))
    cos_projthR_cos_LF.append(MFPAD_R_cos_LF[j].sum(axis=0))
    phi_projthR_cos_LF.append(MFPAD_R_cos_LF[j].sum(axis=1))

#conversion into numpy array
cos_projthS_phi_LF=np.array(cos_projthS_phi_LF)
phi_projthS_phi_LF=np.array(phi_projthS_phi_LF)
cos_projthR_phi_LF=np.array(cos_projthR_phi_LF)
phi_projthR_phi_LF=np.array(phi_projthR_phi_LF)

cos_projthS_cos_LF=np.array(cos_projthS_cos_LF)
phi_projthS_cos_LF=np.array(phi_projthS_cos_LF)
cos_projthR_cos_LF=np.array(cos_projthR_cos_LF)
phi_projthR_cos_LF=np.array(phi_projthR_cos_LF)

#////////////////////////////// NORM \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
MFPAD_S_cos_LF_norm=[]
MFPAD_S_phi_LF_norm=[]
MFPAD_R_cos_LF_norm=[]
MFPAD_R_phi_LF_norm=[]
mPECD_LF_cos=[]
mPECD_LF_phi=[]

#///////////////////////// cos ///////////////////////////////////
for cols,colr in zip(MFPAD_S_cos_LF,MFPAD_R_cos_LF):
    MFPAD_S_cos_LF_norm.append(cols/cols.sum(axis=1)[:,np.newaxis])
    MFPAD_R_cos_LF_norm.append(colr/colr.sum(axis=1)[:,np.newaxis])

MFPAD_S_cos_LF_norm=np.array(MFPAD_S_cos_LF_norm)
MFPAD_R_cos_LF_norm=np.array(MFPAD_R_cos_LF_norm)
MFPAD_S_cos_LF_norm_sum=np.sum(MFPAD_S_cos_LF_norm,axis=0)
MFPAD_R_cos_LF_norm_sum=np.sum(MFPAD_R_cos_LF_norm,axis=0)

mPECD_LF_cos=np.divide(np.subtract(MFPAD_R_cos_LF_norm,MFPAD_S_cos_LF_norm),np.add(MFPAD_R_cos_LF_norm,MFPAD_S_cos_LF_norm))
mPECD_LF_cos_sum=np.divide(np.subtract(MFPAD_R_cos_LF_norm_sum,MFPAD_S_cos_LF_norm_sum),np.add(MFPAD_R_cos_LF_norm_sum,MFPAD_S_cos_LF_norm_sum))

#///////////////////////// phi ///////////////////////////////////
for colsp,colrp in zip(MFPAD_S_phi_LF,MFPAD_R_phi_LF):
    MFPAD_S_phi_LF_norm.append(colsp/colsp.sum(axis=1)[:,np.newaxis])
    MFPAD_R_phi_LF_norm.append(colrp/colrp.sum(axis=1)[:,np.newaxis])

MFPAD_S_phi_LF_norm=np.array(MFPAD_S_phi_LF_norm)
MFPAD_R_phi_LF_norm=np.array(MFPAD_R_phi_LF_norm)
MFPAD_S_phi_LF_norm_sum=np.sum(MFPAD_S_phi_LF_norm,axis=0)
MFPAD_R_phi_LF_norm_sum=np.sum(MFPAD_R_phi_LF_norm,axis=0)

#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
mPECD_LF_phi=np.divide(np.subtract(MFPAD_R_phi_LF_norm,MFPAD_S_phi_LF_norm),np.add(MFPAD_R_phi_LF_norm,MFPAD_S_phi_LF_norm))
mPECD_LF_phi_sum=np.divide(np.subtract(MFPAD_R_phi_LF_norm_sum,MFPAD_S_phi_LF_norm_sum),np.add(MFPAD_R_phi_LF_norm_sum,MFPAD_S_phi_LF_norm_sum))

#NOTE: R-S nad S-R seems to have no influence on b1 map
cPECD_LF_cos =  np.divide(np.subtract(cos_projthR_cos_LF,cos_projthS_cos_LF),np.add(cos_projthR_cos_LF,cos_projthS_cos_LF))
cPECD_LF_phi =  np.divide(np.subtract(cos_projthR_phi_LF,cos_projthS_phi_LF),np.add(cos_projthR_phi_LF,cos_projthS_phi_LF))
#it is somehow unconventional, but let's normalise the error with the square root of all counts
errPECD_LF_cos = np.sqrt(1/(cos_projthR_cos_LF+cos_projthS_cos_LF))

In [ ]:
MFPAD_S_phi_LF,ctheta_test,phi_test=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad,phi_rad,cosphi_adj_phi)

In [ ]:
el = MFPAD_S_phi[0].reshape(-1)
e0mom_x_MF = el * np.sin(theta_rad) * np.cos(phi_rad)
e0mom_y_MF = el * np.sin(theta_rad) * np.sin(phi_rad)
e0mom_z_MF = el * np.cos(theta_rad)

xyzm = np.stack((e0mom_x_MF, e0mom_y_MF, e0mom_z_MF))

# e0mom_x_LF, e0mom_y_LF, e0mom_z_LF = np.einsum('ik, kj -> ij', rot3d(np.arccos(-0.15),1.14,0), xyzm)
e0mom_x_LF, e0mom_y_LF, e0mom_z_LF = np.einsum('ik, kj -> ij', fugi.rot3d(np.arccos(-0.167),15.*np.pi/180.,0.), xyzm)
e0mom_LF = np.sqrt(e0mom_x_LF**2+e0mom_y_LF**2+e0mom_z_LF**2)
      
r_test0=(e0mom_LF)
ctheta_test0=(e0mom_z_LF/e0mom_LF)
#atan2 domain  −π < θ ≤ π
phi_test0=np.arctan2(e0mom_y_LF,e0mom_x_LF)*180./np.pi


fig1, ax = plt.subplots(1,4, figsize=(40,10), constrained_layout=True)
# fig1, ax = plt.subplots(constrained_layout=True)

# CS = ax.contourf(phi_test[0],ctheta_test[0],MFPAD_S_phi_LF[0], levels=20)
# CS = ax.contourf(MFPAD_S_phi_LF[0], levels=20)
ax[0].scatter(phi_full,ctheta_full)
ax[0].set_xlabel('phi [DEG]')
ax[0].set_ylabel('cos(theta) [adm]')
ax[0].set_title("Sen_1_1_initial")

ax[1].scatter(phi_test0,ctheta_test0)
ax[1].set_xlabel('phi [DEG]')
ax[1].set_ylabel('cos(theta) [adm]')
ax[1].set_title("Sen_1_1_rot")

ax[2].contourf(np.array(r_test0).reshape(200,100))
ax[2].set_xlabel('phi [DEG]')
ax[2].set_ylabel('cos(theta) [adm]')
ax[2].set_title("MFPAD_Sen_1_1_nocosphi")

ax[3].contourf(phi_test0.reshape(200,100), ctheta_test0.reshape(200,100), r_test0.reshape(200,100))
ax[3].set_xlabel('phi [DEG]')
ax[3].set_ylabel('cos(theta) [adm]')
ax[3].set_title("MFPAD_Sen_1_1_rot")


fig1.colorbar(CS)

In [ ]:
plt.plot(e0mom_z_LF,e0mom_LF)

In [ ]:
print(xyzm[1])
fugi.rot3d(np.arccos(-0.15),-180*np.pi/180.,0)

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)

# fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    #ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_cos_LF[i].reshape(200,100), levels=20, vmin=0.0001)
    # ax.contourf(MFPAD_S_cos_LF[i].T, levels=20, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], vmin=0.0001)
    ax.contourf(phi_test[i],ctheta_test[i],MFPAD_S_phi_LF[i], levels=20)
    ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

newax = fig2a.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig2a.tight_layout()
plt.show()    

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)

# fig2, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig2.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_cos[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_cos[i].reshape(200,100), levels=20)
    ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

newax = fig2.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig2.tight_layout()
plt.show()    

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
#S ENANTIOMER
# all the 72 plots: no borders & sharing axis

fig3, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row') #figsize in inche by defauls: 1 in = 2.8 cm
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row') #downsized
fig3.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)

newax = fig3.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig3.tight_layout()
plt.show()

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
# fig4, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig4, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig4.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_R_cos[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.text(0, 0, "R_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)

newax = fig4.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig4.tight_layout()
plt.show()

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
fig4a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig4a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig4a.subplots_adjust(hspace = .5, wspace=.5)

for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_R_phi[i].reshape(200,100), levels=20, vmin=0.0001)

newax = fig4a.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig4a.tight_layout()
plt.show()

In [ ]:
#TEST of the PECD because of manual inversion to R enantiomer
print(np.sum(cos_projthR_cos[60]))
plt.plot(errPECD_cos[60])
plt.plot(cPECD_cos[60])
plt.plot(cos_projthR_cos[60])
plt.plot(cos_projthS_cos[60])

In [ ]:
plt.plot(cos_projthR_cos[0])
plt.plot(cos_projthS_cos[0])

In [ ]:
# interpolation modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30*x**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_cos = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)
# custom_ylim = (-1, 1)

for i,ax in zip(range(72),axes.flatten()): #72 fixed because of the photon directions, equal to range(len(cosphi))
    x_data = np.flip(cosM)
    y_data = cPECD_cos[i]
    y_err = errPECD_cos[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator

    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    #NOTE: x_data has to be monotonicalli INCREASING!
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax.plot(x_data_new,bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix_cos[j][i] = ((m.values[p], m.errors[p]))
#numpy tranformation!!!
param_matrix_cos=np.array(param_matrix_cos) 

newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
print(param_matrix_cos.shape)
param_matrix_cos[0,:,0].shape
param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[0,:,0], x=go_phi, y=go_cos, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(param_matrix_cos)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_cos[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_cos[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))
errPECD_phi = np.sqrt(1/(cos_projthR_phi+cos_projthS_phi))

In [ ]:
cosM

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_phi = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)
# custom_ylim = (-1, 1)

for i,ax in zip(range(72),axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    #print(i)
    x_data = np.flip(cosM)
    y_data = cPECD_phi[i]
    y_err = errPECD_phi[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator

    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax.plot(x_data_new, bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix_phi[j][i] = ((m.values[p], m.errors[p]))

#numpy tranformation!!!
param_matrix_phi=np.array(param_matrix_phi) 

newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[0,:,0], x=go_phi, y=go_cos, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(param_matrix_phi)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#INTEGRATED S AND R MFPADs
#contour

fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True,figsize=(12, 6))

ax0.contourf(phiM,cosM,MFPAD_R_phi_norm_sum.reshape(200,100).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]])
ax0.set_title("MFPAD R enant")
ax0.set_xlabel('phi [DEG]')
ax0.set_ylabel('cos(theta) [adm]')

ax1.contourf(phiM,cosM,MFPAD_S_phi_norm_sum.reshape(200,100).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]])
ax1.set_title("MFPAD S enant")
ax1.set_xlabel('phi [DEG]')

plt.show()

In [ ]:
#INTEGRATED S AND R MFPADs
#NOTE: the phi sorted is the same as it should be
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True, sharey=True,figsize=(12, 6))

ax0.imshow(MFPAD_R_cos_norm_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap='viridis', aspect="auto")
ax0.set_title("MFPAD R enant")
ax0.set_xlabel('phi [DEG]')
ax0.set_ylabel('cos(theta) [adm]')

ax1.imshow(MFPAD_S_cos_norm_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap='viridis', aspect="auto")
ax1.set_title("MFPAD S enant")
ax1.set_xlabel('phi [DEG]')

plt.show()

In [ ]:
cosM

In [ ]:
#NOTE: has to be transposed

# interpolation: [ 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'.]

fig0=plt.figure()
plt.contourf(phiM,cosM,mPECD_cos_sum.T, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], cmap=cmap_temp)
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
plt.title("contour theory")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

fig1=plt.figure()
plt.imshow(mPECD_cos_sum.T, vmin=-0.3, vmax=0.3, origin="lower", extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
plt.title("imshow theory")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

fig2=plt.figure()
plt.imshow(mPECD_cos_sum.T, vmin=-0.3, vmax=0.3, origin="lower", extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='bilinear', cmap=cmap_temp, aspect="auto")
plt.title("imshow theory bilinear")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

fig3=plt.figure()
plt.imshow(mPECD_cos_sum.T, vmin=-0.3, vmax=0.3, origin="lower", extent=[phiM[0],phiM[-1],cosM[-1],cosM[0]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
plt.title("imshow theory quadratic interpolation")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

